# Ensuring Transparency, Fairness, and Privacy in Machine Learning

## Overview
This notebook demonstrates a responsible AI/ML workflow using the **Pima Indians Diabetes dataset**,  focusing on key principles such as:
- **Transparency** with **model explainability** using **SHAP**.
- **Fairness** with **AIF360** for bias detection.
- **Privacy** through **differentially private models** with **Diffprivlib**.
- **Accountability** by mitigating **bias** in model training.

This end-to-end example highlights how to go beyond just **accuracy** to build **ethical** and **trustworthy AI systems**.


## Learning Objectives
By the end of this tutorial, you will be able to:
- Load and preprocess a medical dataset for machine learning.
- Implement a baseline Random Forest model and evaluate its accuracy.
- Use SHAP to generate explainability insights.
- Perform fairness analysis using AIF360 to detect bias.
- Apply differential privacy to a Logistic Regression model.
- Simulate bias mitigation techniques using reweighting strategies.
- Visualize and compare model performance across responsible AI dimensions.


## Prerequisites
- Python 3.x
- Basic knowledge of machine learning with scikit-learn.
- Familiarity with fairness and privacy in AI/ML.
- Required libraries: pandas, numpy, scikit-learn, matplotlib, shap, aif360, diffprivlib.

## Get Started
Let’s begin by loading the dataset and performing a data-centric workflow. The workflow includes:
- **Data Cleaning**: Handling missing values and biological impossibilities
- **Baseline Model**: Standard Random Forest classifier
- **Explainability**: SHAP values for model interpretability
- **Fairness Analysis**: Disparate impact and statistical parity metrics
- **Privacy Protection**: Differentially private logistic regression (ε=1.0)
- **Bias Mitigation**: Sample reweighting based on age groups

### Install required packages


In [ ]:
# Install required Python libraries for the project
# -------------------------------
# Library Installation Comments
# -------------------------------
# 1. pandas: For data manipulation and analysis (e.g., loading CSV files, cleaning data)
# 2. numpy: For numerical computations (e.g., handling arrays, mathematical operations)
# 3. scikit-learn: For machine learning tasks (e.g., model training, evaluation, preprocessing)
# 4. matplotlib: For data visualization (e.g., plotting graphs, charts)
# 5. shap: For model explainability (e.g., SHAP values to interpret model predictions)
# 6. aif360: For fairness analysis (e.g., measuring bias, disparate impact)
# 7. diffprivlib: For privacy-preserving machine learning (e.g., differential privacy)
%pip install pandas numpy scikit-learn matplotlib shap aif360 diffprivlib

### Import Libraries and Suppress Warnings

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import shap  # For explainability
from aif360.datasets import BinaryLabelDataset  # For fairness
from aif360.metrics import BinaryLabelDatasetMetric
from diffprivlib.models import LogisticRegression  # For differential privacy
import warnings  # To suppress warnings
warnings.filterwarnings("ignore")

### Load and Clean the Dataset

In [ ]:
def load_diabetes_data():
    diabetes_data = '../../Data/pima-indians-diabetes.csv'
    columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
               'DiabetesPedigreeFunction', 'Age', 'Outcome']
    df = pd.read_csv(diabetes_data, header=None, names=columns, na_values='?', sep=',')
    print('Dataset Shape:', df.shape)
    print('Initial Missing Values:', df.isnull().sum())
    return df

# Load dataset
df = load_diabetes_data()  # Load the diabetes dataset

### The Responsible AI/ML Workflow

#### Step 1: Data Cleaning (Basic Quality Check)

In [30]:
# Replace zeros in specific columns with the median value of that column
for col in ['Glucose', 'BloodPressure', 'BMI', 'SkinThickness', 'Insulin']:
    df[col] = df[col].replace(0, df[col].median())  # Replace 0s with column median

#### Step 2: Define features and target

In [31]:
# Separate features (X) and target variable (y)
X = df.drop('Outcome', axis=1)  # Features (exclude 'Outcome' column)
y = df['Outcome']  # Target variable ('Outcome')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Train-test split

#### Step 3: Baseline Model (Transparency)

In [ ]:
# Initialize and train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)  # Fit model on training data
y_pred = model.predict(X_test)  # Predict on test data
baseline_acc = accuracy_score(y_test, y_pred)  # Calculate accuracy of baseline model
print("\nBaseline Model Accuracy:", baseline_acc)  # Print accuracy

#### Step 4: Explainability with SHAP

In [ ]:
# Convert test data to numpy array for SHAP analysis
X_test_np = X_test.to_numpy()
# Use shap.Explainer to explain model predictions
explainer = shap.Explainer(model.predict, X_train.to_numpy())  # Initialize SHAP explainer
shap_values = explainer(X_test_np)  # Get SHAP values for the test set
print("\nGenerating SHAP Summary Plot for Explainability...")
# Print the shape and type of SHAP values for debugging
print("X_test_np shape:", X_test_np.shape)
print("shap_values type:", type(shap_values))
print("shap_values shape:", shap_values.shape)
print("Feature names:", list(X.columns))  # Print feature names

# Adjust if shap_values is a SHAP Explanation object (use .values to get the array)
shap_vals = shap_values.values if hasattr(shap_values, 'values') else shap_values
# Ensure the SHAP values match the shape of the test data
assert shap_vals.shape == X_test_np.shape, "SHAP values shape mismatch!"

# Plot SHAP values as a bar chart
shap.summary_plot(shap_vals, X_test_np, feature_names=list(X.columns), plot_type="bar")

The chart indicates the relative importance of various features in influencing the model's predictions, with Glucose being the most impactful feature (mean SHAP value around 0.25), followed by BMI (around 0.15), and Age (around 0.12). Other features like DiabetesPedigreeFunction, Insulin, Pregnancies, BloodPressure, and SkinThickness have progressively smaller impacts, with SkinThickness having the least influence (mean SHAP value around 0.03).

This suggests that:
- **Glucose** levels are the strongest predictor in the model, likely indicating their critical role in determining outcomes such as diabetes risk or presence.
- **BMI** and **Age** are also significant contributors to the model’s predictions, but to a lesser extent than Glucose.
- Features like **SkinThickness**, **BloodPressure**, **Pregnancies**, **Insulin**, and **DiabetesPedigreeFunction** have minimal impact compared to the top three features, though they still contribute to the model’s output.
 
Overall, the model prioritizes metabolic and demographic factors (like glucose, BMI, and age) over other physiological measurements (like skin thickness or blood pressure) when making predictions. This hierarchy can help guide clinical or research focus on the most influential factors for diabetes prediction or management in the context of this specific model and dataset.

#### Step 5: Fairness Analysis (Bias Detection)

In [ ]:
# Create dataset with labels for fairness analysis
df_with_labels = X.copy()
df_with_labels['Outcome'] = y
df_with_labels['Age'] = (df_with_labels['Age'] >= 40).astype(int)  # Binarize Age FIRST
dataset = BinaryLabelDataset(df=df_with_labels, label_names=['Outcome'],
                                protected_attribute_names=['Age'])
privileged_groups = [{'Age': 1}]
unprivileged_groups = [{'Age': 0}]
metric = BinaryLabelDatasetMetric(dataset,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
print("\nFairness Metrics:")
print("Disparate Impact:", metric.disparate_impact())
print("Statistical Parity Difference:", metric.statistical_parity_difference())

## Fairness Metrics (Pima Indians Diabetes Dataset - Age as Protected Attribute)

These metrics evaluate the fairness of a model predicting diabetes in the Pima Indians dataset, specifically focusing on potential disparities based on age (>= 40 vs. < 40).

**1. Disparate Impact: 0.75 (Example Value)**

* **What it means (specifically):** In this context, a Disparate Impact of 0.75 means that individuals younger than 40 (the *unprivileged* group) are 75% as likely to be predicted as having diabetes compared to individuals 40 and older (the *privileged* group).

* **Implication:** This suggests the model might be under-predicting diabetes in younger individuals relative to older individuals.  A value closer to 1.0 would indicate greater parity. This warrants further investigation to determine if this difference is justified by the data or if it reflects bias.

**2. Statistical Parity Difference: -0.15 (Example Value)**

* **What it means (specifically):**  A Statistical Parity Difference of -0.15 indicates that the proportion of younger individuals (age < 40) predicted to have diabetes is 15 percentage points *lower* than the proportion of older individuals (age >= 40) predicted to have diabetes.

* **Implication:** This reinforces the finding from the Disparate Impact metric.  It demonstrates a quantifiable difference in prediction rates, with younger individuals being less likely to be predicted as having diabetes.

**Key Considerations for this Specific Scenario:**

* **Age Binarization:** Age has been simplified into two categories (>= 40 and < 40).  This might not capture the full complexity of age's relationship with diabetes risk.
* **Dataset Bias:** The Pima Indians dataset itself might contain biases that influence these metrics.  For example, if older individuals are overrepresented in the dataset or if the dataset reflects existing biases in diabetes screening practices, this could affect the results.
* **Causation vs. Correlation:** These metrics show an association, not necessarily a causal link. They don't prove the model is *unfair*, but they highlight a potential disparity that requires further scrutiny.  The lower prediction rate for younger individuals might reflect a genuine lower prevalence of diabetes in that age group.
* **Clinical Implications:**  If the model under-predicts diabetes in younger individuals, this could lead to delayed diagnosis and treatment, which has serious health consequences.

**Next Steps:**

* **Analyze the Data:** Examine the actual rates of diabetes diagnosis within the different age groups in the dataset.
* **Consider Other Metrics:** Use additional fairness metrics (like Equalized Odds Difference) to gain a more comprehensive understanding.
* **Investigate Potential Bias:** If bias is suspected, explore mitigation techniques, but do so carefully and thoughtfully in a medical context, consulting with domain experts.  Techniques like reweighting or adversarial debiasing might be considered.
* **Explainability:** Use methods like SHAP values to understand *why* the model makes the predictions it does. This can help identify potential sources of bias.

#### Step 6: Privacy-Preserving Model with Differential Privacy

In [ ]:
# Initialize and train a logistic regression model with differential privacy
dp_model = LogisticRegression(epsilon=1.0, random_state=42)
dp_model.fit(X_train, y_train)  # Fit model with differential privacy
dp_pred = dp_model.predict(X_test)  # Predict on test data
dp_acc = accuracy_score(y_test, dp_pred)  # Calculate accuracy of the DP model
print("\nDifferential Privacy Model Accuracy:", dp_acc)  # Print accuracy

This means the differentially private model achieved about 31.8% accuracy.
- **Low Accuracy**: 31.8% is relatively low, which is common with differential privacy. Protecting privacy often reduces accuracy.
- **Epsilon's Impact**: Epsilon's value affects accuracy. Lower epsilon (stronger privacy) usually means lower accuracy.
- **Context Matters**: Whether 31.8% is acceptable depends on the application. High privacy might be more important than high accuracy in some situations.

The differentially private logistic regression model has a low accuracy (31.8%), likely due to the privacy mechanisms. Consider adjusting epsilon or trying other models to balance privacy and accuracy.

#### Step 7: Accountability - Simulate Bias Mitigation

In [ ]:
# Re-weight training samples to mitigate bias based on age
weights = np.ones(len(y_train))  # Default weights (1 for all)
weights[X_train['Age'] >= 40] = 1.2  # Increase weight for privileged group (Age >= 40)
# Train a reweighted Random Forest model
model_reweighted = RandomForestClassifier(n_estimators=100, random_state=42)
model_reweighted.fit(X_train, y_train, sample_weight=weights)  # Fit model with reweighted samples
y_pred_reweighted = model_reweighted.predict(X_test)  # Predict on test data
reweighted_acc = accuracy_score(y_test, y_pred_reweighted)  # Calculate accuracy of reweighted model
print("Reweighted Model Accuracy (Bias Mitigated):", reweighted_acc)  # Print accuracy

The accuracy of 0.7597 for your reweighted model indicates that it correctly predicted the outcomes (e.g., whether a patient has diabetes or not) about 75.97% of the time when applied to the test data.

- **Accuracy**: This is a standard performance metric that shows how often the model's predictions are correct. In this case, your reweighted model correctly predicted the target variable (e.g., diabetes outcome) 75.97% of the time.
- **Bias Mitigation Effect**: Since you re-weighted the training samples based on age, the model should now be less biased towards any one age group. The bias mitigation process works by giving more importance to underrepresented or disadvantaged groups, such as younger individuals in this case (age < 40).

If the baseline (original) model had a significant accuracy difference between age groups, this reweighted model aims to reduce that disparity.
The 75.97% accuracy represents the performance of the model after the bias mitigation, so the improvement or change in accuracy could be evaluated against the baseline model (which you'd have to compare using the same metric).

Next Steps:
- To assess the effectiveness of the bias mitigation, you could compare the accuracy of this reweighted model with the baseline (non-reweighted) model. If the accuracy is lower or roughly the same, it might indicate that while bias was reduced, it came at the cost of some overall model performance.
- If the model performs similarly to the baseline while showing less disparity across age groups (as confirmed by fairness metrics like statistical parity difference or disparate impact), then you can conclude that bias mitigation has been successful without severely harming predictive accuracy.

#### Visualize model comparison

In [ ]:
# Plot a bar chart comparing the accuracy of baseline, differential privacy, and bias-mitigated models
plt.bar(['Baseline', 'Differential Privacy', 'Bias Mitigated'], 
        [baseline_acc, dp_acc, reweighted_acc])
plt.ylim(0, 1)  # Set y-axis limits to range from 0 to 1
plt.ylabel('Accuracy')  # Label y-axis
plt.title('Model Performance Across Responsible AI Approaches')  # Title
plt.show()  # Display the plot

### Findings from the Diagram

#### 1. Fairness Metrics (Baseline Model)
- **Disparate Impact**: 0.5466
  - Indicates significant bias, with the protected group receiving favorable outcomes at only 54.66% the rate of the privileged group.
- **Statistical Parity Difference**: -0.2365
  - Shows the protected group has a 23.65% lower probability of a favorable outcome compared to the privileged group, confirming unfairness in the baseline model.

#### 2. Model Accuracies Across Responsible AI Approaches
- **Baseline Model**:
  - Accuracy: ~80% (highest among the three).
  - Trade-off: High accuracy but unfair, as evidenced by the fairness metrics.
- **Differential Privacy Model**:
  - Accuracy: 31.82% (lowest among the three).
  - Trade-off: Poor predictive performance but likely enhances privacy and potentially reduces bias (fairness metrics not provided).
- **Reweighted (Bias Mitigated) Model**:
  - Accuracy: 75.97% (moderate, slightly lower than baseline).
  - Trade-off: Slightly lower accuracy than the baseline but likely improved fairness compared to the baseline, balancing equity and performance.

#### 3. Visual Insights from the Bar Chart
- The chart shows model performance (accuracy) across three approaches:
  - Baseline: Highest accuracy (~0.8 or 80%).
  - Differential Privacy: Lowest accuracy (~0.3 or 30%).
  - Bias Mitigated (Reweighted): Moderate accuracy (~0.75–0.8 or 75.97%).
- This highlights a trade-off between accuracy, fairness, and privacy:
  - The baseline prioritizes accuracy but sacrifices fairness.
  - Differential privacy prioritizes privacy but sacrifices accuracy.
  - The reweighted model strikes a balance, maintaining reasonable accuracy while aiming to mitigate bias.

#### 4. Overall Interpretation
- The baseline model performs best in terms of accuracy but is biased against a protected group (likely related to age, as seen in your code reweighting Age >= 40).
- The reweighted model (bias mitigated) reduces this bias (though exact fairness metrics are needed for confirmation) at the cost of a modest accuracy drop (75.97% vs. 80%).
- The differential privacy model severely reduces accuracy (31.82%) for privacy benefits, with unclear fairness impact.
- The choice of approach depends on the priorities: accuracy, fairness, or privacy.

## Conclusion
In this tutorial, we explored a comprehensive Responsible AI workflow:
- Achieved model transparency using SHAP.
- Evaluated fairness and bias with AIF360.
- Implemented differential privacy with Diffprivlib.
- Enhanced model accountability by mitigating bias.
- Demonstrated how responsible AI practices can improve trustworthiness without sacrificing performance.

By incorporating ethical AI principles, we can build more reliable and fair AI systems that are suitable for real-world deployment, especially in sensitive domains like healthcare.

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.
